In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [ ]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-20 05:15:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.17MB/s    in 0.2s    

2022-06-20 05:15:21 (5.17 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon_reviews_us_Watches").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep="\t")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
# Check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Change review_date to an actual date
from pyspark.sql.types import DateType

df = df.withColumn("review_date", df.review_date.cast(DateType()))
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
# Drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

960872
960679
960679


In [ ]:
# Check data types after changes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Create review_id_table
reviews_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000IUWCC6GL8|   47798261|B0019R9IME|     793693894| 2010-04-16|
|R100JWMI1NI5CL|     812108|B00284C9U0|     313836033| 2015-01-02|
|R100MLMPIBZ7CG|   32189911|B006GEQBPW|     319438143| 2014-02-26|
|R101BVFO3458UC|   34404699|B002OSY3WM|     966359187| 2013-11-14|
|R101Q0ATHRL4RA|   51408517|B002X7AAI0|     207535892| 2014-10-09|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Create products table
products_df = df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0019R9IME|Michael Kors Wome...|
|B00284C9U0|Timex Men's T4971...|
|B006GEQBPW|22mm Grey Canvas ...|
|B002OSY3WM|Casio Men's PAW20...|
|B002X7AAI0|Oakley Men's 10-0...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Count the total number of rows in products_df
counted = products_df.count()
print(f"Total number before: {counted}")

Total number before: 960679


In [ ]:
# Only keep distinct products
products_df = products_df.distinct()

# Check the count now
recounted = products_df.count()
print(f"Total number after: {recounted}")

Total number after: 149694


In [ ]:
# Create customers table
# Per the schema, create a count of customers
customers_df = df.groupBy("customer_id").count()

# Rename count column
customers_df = customers_df.withColumnRenamed("count", "customer_count")

# Order by customer count
customers_df.orderBy(customers_df["customer_count"].desc()).show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42418272|           407|
|   40765068|           127|
|   19619134|           121|
|   52460875|           111|
|    5072809|           106|
|   18691646|            92|
|   14726863|            88|
|   13355404|            87|
|   42416004|            86|
|   45112699|            75|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Check data types for customers_df
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
from pyspark.sql.types import IntegerType

# Per the schema, convert customer_count to integer
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))

# Check data types after changes
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# Create vine_table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000IUWCC6GL8|          5|            9|         11|   N|
|R100JWMI1NI5CL|          3|            0|          0|   N|
|R100MLMPIBZ7CG|          2|            0|          0|   N|
|R101BVFO3458UC|          2|            0|          2|   N|
|R101Q0ATHRL4RA|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<CONNECTION STRING>:5432/<DATABASE>"
config = {"user":"<USERNAME>", 
          "password": "<PASSWORD>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataframe to review_id_table table in RDS
reviews_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

NameError: ignored

In [ ]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)